In [1]:
from graph import GraphDB

import random
import numpy as np
import re
import pandas as pd
from ipywidgets import interact, widgets
import pandas as pd
from datetime import datetime

C:\Anaconda\envs\ds_env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda\envs\ds_env\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Anaconda\envs\ds_env\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# getting data 
date_data_path = r'../data/tender_postcode.xlsx'

path = r'../data/embedding_data/'
embedding_data = np.load(path+"sent_transformer_embeddings.npz")
embedding_data = [list(embedding_data[f]) for f in embedding_data.files]

tender_references_raw = ''
tender_titles_raw = ''

with open(path+'tender_references.txt', 'r') as file:
    for line in file:
        tender_references_raw += line

# getting a few encoding errors, unlikely to be an issue
with open(path+"tender_titles.txt",'r', errors = "ignore") as file:
    for line in file:
        tender_titles_raw += line
        
        
for item in ['"',"'",'[',']'," "]:
    tender_references_raw = tender_references_raw.replace(item, "")
tender_references = tender_references_raw.split(",")

tender_titles = []
for item in tender_titles_raw.split(","):
    tender_titles.append(re.sub(r'[\[\]/\\\'"]', '', item).lstrip())

# load clustering data    
topic_data = pd.read_csv("../data/clustering/sentence_bert_clusters.csv")
topic_data_list = list(topic_data['Cluster'])
cluster_data = pd.read_csv("../data/clustering/sentence_bert_cluster_topics.csv")
topics_list = list(cluster_data['Topics'])

coordinate_data = np.load(path+"tender_coordinates.npz")
coordinate_data = [list(coordinate_data[f]) for f in coordinate_data]

expire_dates = pd.read_excel(date_data_path)
expire_dates = list(expire_dates['Expiry Date'])

In [3]:
# save to dictionary format
embedding_dictionary = {}
for i in range(len(tender_references)):
    embedding_dictionary[tender_references[i]] = {"embedding":embedding_data[i],"titles":tender_titles[i],"location":coordinate_data[i],"expire_date":expire_dates[i],"cluster":topic_data_list[i],'topics':topics_list[topic_data_list[i]]}

In [4]:
graph_db = GraphDB(embedding_dictionary)

<h2>Querying graph database by reference <h2>

This section allows for querying of the graph database by reference number or reference title, along with several options for tweaking the relationships that are returned. The parameter search options are: 

Reference: This textbox allows you to query the database, and look at tenders that are similar to it. It accepts a reference id, or reference title. 

Closest Tenders: This slider allows you to specify the n closest tenders to include in the map view or network view. 

Max Distance: This slide allows you to restrict which tenders are included in the query results by the distance in kilometres. 

Level of Neighbours: This slider allows you to include the neighbours of nodes similar to your id or title entered in the reference textbox, increasing the value of the slider increases the level of neighbours to travel down




In [14]:
start_date = datetime(2018, 1, 1)
end_date = datetime(2036, 1, 1)

dates = pd.date_range(start_date, end_date, freq='M')

options = [(date.strftime(' %b %Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Tender Expiration date ',
    orientation='horizontal',
    layout={'width': '500px'}
)
tender_neighbour_slider = widgets.IntSlider(value=1, min=0, max=10, step=1, description='Level of Neighbours:')

reference_input = widgets.Text(value='Please enter a value', description='Reference :   ')
number_slider = widgets.IntSlider(value=2, min=1, max=50, step=1, description='Closest Tenders:')
distance_slider = widgets.FloatSlider(value=100, min=1, max=300, step=1, description='Max Distance:')

tender_expire_slider = widgets.FloatSlider(value=100, min=1, max=300, step=1, description='Tender Expiry Month:')

topics_search = widgets.Checkbox(value=False,description='topics',disabled=False,indent=False)
topic_input = widgets.Text(value='Please enter a value', description='Enter topic number :')
topics_search.layout.visibility, topic_input.layout.visibility = "hidden",'hidden'
topics_search.layout.display,topic_input.layout.display = "none","none"
interact(graph_db.query_and_display, reference=reference_input, num_nodes=number_slider, max_distance=distance_slider,expire_date=selection_range_slider,neighbours = tender_neighbour_slider,topic=topic_input,topic_box=topics_search)

interactive(children=(Text(value='Please enter a value', description='Reference :   '), IntSlider(value=2, des…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

<h2>Querying graph database by topic number<h2>

This section allows you to search for tenders by topic similarity. There is only one interactable element here, the topic number textbox. Enter a topic number to see all tenders that are included under that topic.

In [13]:
topics_search = widgets.Checkbox(value=True,description='topics',disabled=False,indent=False)
topic_input = widgets.Text(value='Please enter a value', description='topic number :')
topics_search.layout.display = 'none'
selection_range_slider.layout.display, tender_neighbour_slider.layout.visibility, reference_input.layout.visibility,number_slider.layout.visibility,distance_slider.layout.visibility,tender_expire_slider.layout.visibility = "hidden",'hidden','hidden','hidden','hidden','hidden'
selection_range_slider.layout.display,tender_neighbour_slider.layout.display, reference_input.layout.display,number_slider.layout.display,distance_slider.layout.display,tender_expire_slider.layout.display = "none","none","none","none","none","none",


interact(graph_db.query_and_display, reference=reference_input, num_nodes=number_slider, max_distance=distance_slider,expire_date=selection_range_slider,neighbours = tender_neighbour_slider,topic=topic_input,topic_box=topics_search)

interactive(children=(Text(value='Please enter a value', description='Reference :   ', layout=Layout(display='…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [15]:
list(embedding_dictionary.keys())[0]

'ARA201901891'